# FiftyOne games

Playing with [FiftyOne](https://voxel51.com/docs/fiftyone/) to:

* Download the fish subset of [OpenImages](https://storage.googleapis.com/openimages/web/download.html), with the goal of eventually improving on the off-the-shelf [fish detection](https://github.com/rocapal/fish_detection)
* Learn how to use the app to work with my datasets, and potentially use it for validating batch annotation on trips


# TODO

* Import Eviota and visualise fish / no fish + classifications from other notebook

In [47]:
import json
from pathlib import Path

import fiftyone as fo
import fiftyone.utils.yolo as fouy
import fiftyone.zoo as foz

In [6]:
dataset = foz.load_zoo_dataset(
    "open-images-v6",
    label_types=["detections"],
    classes=["Fish", "Goldfish"],
)

 100% |██████|    4.8Gb/4.8Gb [3.0m elapsed, 0s remaining, 41.9Mb/s]      
Connection pool is full, discarding connection: open-images-dataset.s3.amazonaws.com
 100% |█████████████████| 5841/5841 [11.7m elapsed, 0s remaining, 2.5 files/s]      
 100% |█████████████████| 1011/1011 [2.2m elapsed, 0s remaining, 11.7 files/s]      
 100% |███████████████████| 370/370 [43.9s elapsed, 0s remaining, 8.1 files/s]       
Dataset info written to '/home/yanir/fiftyone/open-images-v6/info.json'
Loading 'open-images-v6' split 'train'
 100% |███████████████| 5841/5841 [23.6s elapsed, 0s remaining, 256.4 samples/s]      
Loading 'open-images-v6' split 'test'
 100% |███████████████| 1011/1011 [3.2s elapsed, 0s remaining, 316.9 samples/s]      
Loading 'open-images-v6' split 'validation'
 100% |█████████████████| 370/370 [1.2s elapsed, 0s remaining, 311.8 samples/s]         
Dataset 'open-images-v6' created


In [7]:
dataset

Name:        open-images-v6
Media type:  image
Num samples: 7222
Persistent:  False
Tags:        ['test', 'train', 'validation']
Sample fields:
    id:             fiftyone.core.fields.ObjectIdField
    filepath:       fiftyone.core.fields.StringField
    tags:           fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    detections:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    open_images_id: fiftyone.core.fields.StringField

In [37]:
samples = []
for image_path in Path("/home/yanir/projects/deep-fish/data/eviota-202010/").glob("*.jpg"):
    sample = fo.Sample(filepath=image_path)
    sample["ground_truth"] = fo.Classifications(
        classifications=[
            fo.Classification(label=label.replace('-', ' ').capitalize())
             for label in image_path.name.replace('.jpg', '').split('-', maxsplit=1)[1].split('-AND-')
        ]
    )
    samples.append(sample)

dataset = fo.Dataset("eviota-2020")
dataset.add_samples(samples)

 100% |█████████████████| 533/533 [217.2ms elapsed, 0s remaining, 2.5K samples/s]    


['61ef861f13d4a950c7728d3c',
 '61ef861f13d4a950c7728d3e',
 '61ef861f13d4a950c7728d3f',
 '61ef861f13d4a950c7728d42',
 '61ef861f13d4a950c7728d43',
 '61ef861f13d4a950c7728d44',
 '61ef861f13d4a950c7728d45',
 '61ef861f13d4a950c7728d4a',
 '61ef861f13d4a950c7728d4b',
 '61ef861f13d4a950c7728d4c',
 '61ef861f13d4a950c7728d4d',
 '61ef861f13d4a950c7728d4e',
 '61ef861f13d4a950c7728d4f',
 '61ef861f13d4a950c7728d50',
 '61ef861f13d4a950c7728d51',
 '61ef861f13d4a950c7728d5a',
 '61ef861f13d4a950c7728d5b',
 '61ef861f13d4a950c7728d5c',
 '61ef861f13d4a950c7728d5d',
 '61ef861f13d4a950c7728d5e',
 '61ef861f13d4a950c7728d5f',
 '61ef861f13d4a950c7728d60',
 '61ef861f13d4a950c7728d61',
 '61ef861f13d4a950c7728d62',
 '61ef861f13d4a950c7728d63',
 '61ef861f13d4a950c7728d64',
 '61ef861f13d4a950c7728d65',
 '61ef861f13d4a950c7728d66',
 '61ef861f13d4a950c7728d67',
 '61ef861f13d4a950c7728d68',
 '61ef861f13d4a950c7728d69',
 '61ef862013d4a950c7728d7a',
 '61ef862013d4a950c7728d7b',
 '61ef862013d4a950c7728d7c',
 '61ef862013d4

In [38]:
dataset

Name:        eviota-2020
Media type:  image
Num samples: 533
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classifications)

In [49]:
fouy._parse_yolo_row??

In [52]:
yolov3_pilot_fish_detections = {}
with open("/home/yanir/projects/deep-fish/data/eviota-202010/yolov3-pilot-results.json") as yolov3_fp:
    for frame in json.load(yolov3_fp):
        detections = []
        for bbox in frame["objects"]:
            center_x, center_y, width, height = bbox["relative_coordinates"].values()
            detections.append(
                fo.Detection(
                    label="fish",
                    bounding_box=[center_x - 0.5 * width, center_y - 0.5 * height, width, height],
                    confidence=bbox["confidence"],
                )
            )            
        yolov3_pilot_fish_detections[frame["filename"]] = fo.Detections(detections=detections)

In [54]:
for sample in dataset:
    sample["yolov3_pilot_fish_predictions"] = yolov3_pilot_fish_detections[sample.filepath]
    sample.save()